In [1]:
%matplotlib qt

import os
import pathlib

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import numpy as np

# ExaFMM-T
import exafmm.laplace as laplace

# PyExaFMM
from fmm import Fmm
from fmm.kernel import laplace_p2p_serial, laplace_gradient
from fmm.surface import scale_surface

# Adaptoctree
import adaptoctree.morton as morton
import adaptoctree.tree as tree

# Plotting parameters
plt.rc('font', family='serif', serif='Times')
plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=8)
plt.rc('ytick', labelsize=8)
plt.rc('axes', labelsize=8)

# Dimensions for column plots
# width = 3.487 # half page plot width
width = 4.328 # full page plot width
height = width / 1.618 # golden ratio

HERE = pathlib.Path(os.getcwd())
FIGURE_SAVEPATH = os.path.abspath(HERE.parent.parent / 'article/figures')

## Generate Test Data
Uncomment to generate test data

In [2]:
# ! rm test.hdf5 && fmm generate-test-data -c test && fmm compute-operators -c test
# ! rm C3E3.hdf5 && fmm generate-test-data -c C3E3 && fmm compute-operators -c C3E3
# ! rm C4E4.hdf5 && fmm generate-test-data -c C4E4 && fmm compute-operators -c C4E4
# ! rm C5E5.hdf5 && fmm generate-test-data -c C5E5 && fmm compute-operators -c C5E5
# ! rm C6E6.hdf5 && fmm generate-test-data -c C6E6 && fmm compute-operators -c C6E6
# ! rm C7E7.hdf5 && fmm generate-test-data -c C7E7 && fmm compute-operators -c C7E7

In [9]:
# Compile test experiment to cache numba functions
e = Fmm('test')
e.run()
laplace_p2p_serial(e.sources, e.targets, e.source_densities)
laplace_gradient(e.sources, e.targets, e.source_densities)

array([[ 283.3415 ,  602.8579 ,  448.8763 ],
       [ 728.319  ,  836.0533 ,  150.27887],
       [ 731.808  ,  421.88794,  366.041  ],
       ...,
       [-464.09085, -511.3172 , -427.3405 ],
       [-305.128  , -636.4044 , -557.71857],
       [-272.34122, -617.55475,  -90.45561]], dtype=float32)

## Compare Analyticity of Expansions from PyExaFMM and ExaFMM-T

How does discretisation effect the quality of the expansions? We check the analyticity of multipole expansions by computing the approximated potential at points distributed on the surface of a sphere surrounding the equivalent surface, in the far-field. As the radius of this sphere is increased, we can demonstrate the accuracy of the multipole expansion away from the equivalent surface. We repeat a similar experiment for the local expansions.

In [10]:
def sphere_data(npoints, r, c):
    """
    Generate npoints randomly distributed on the
        surface of a sphere with radius r, and center c
    """
    np.random.seed(0)
    
    phi = np.random.rand(npoints)*2*np.pi
    costheta = (np.random.rand(npoints)-0.5)*2

    theta = np.arccos(costheta)

    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)

    sources = np.vstack((x, y, z))

    return sources.T + c

In [11]:
# test = sphere_data(1000, 10, np.array([10, 10, 0]))

# ax = plt.axes(projection='3d')
# ax.scatter3D(test[:, 0], test[:, 1], test[:, 2], c='k', s=1)
# plt.show()

In [12]:
# Discretisation order of check surface
Cvec = [2, 3, 4, 5, 6, 7]

# Discretisation order of equivalent surface
Evec = [2, 3, 4, 5, 6, 7]

# Load experimental data into PyExaFMM
pyfmmvec = [Fmm(f'C{C}E{C}') for C in Cvec]

In [13]:
# Load experimental data into ExaFMM-T
exafmmvec = []
exafmmtreevec = []

for e in pyfmmvec:
    # create a list of source instances
    sources = laplace.init_sources(e.sources, e.source_densities)

    # create a list of target instances
    targets = laplace.init_targets(e.targets)
    
    # Expansion order
    p = e.config['order_equivalent']
    fmm = laplace.LaplaceFmm(p=p, ncrit=e.config['max_points'], filename=f'C{p}E{p}.dat')
    exafmmvec.append(fmm)
    
    tree = laplace.setup(sources, targets, fmm)
    exafmmtreevec.append(tree)

In [14]:
# Evaluate PyExaFMM experiments
for e in pyfmmvec:
    e.run()

In [15]:
# Evaluate ExaFMM-T experiments
ex_target_potentials = []
for tree, fmm in list(zip(exafmmtreevec, exafmmvec)):
    ex_target_potentials.append(laplace.evaluate(tree, fmm))

### Check convergence of multipole expansion of root node

In [16]:
# Create surfaces
equivalent_surfaces = [
    e.equivalent_surface for e in pyfmmvec
]

upward_equivalent_surfaces = [scale_surface(
    surf=equivalent_surfaces[i], radius=pyfmmvec[i].r0, 
    level=0, center=pyfmmvec[i].x0, alpha=pyfmmvec[i].alpha_outer
) for i in range(len(pyfmmvec))]

# Extract multipole expansion at root node
pyfmm_multipole_expansions = [
    e.multipole_expansions[e.key_to_index[0]:e.key_to_index[0]+e.nequivalent_points]
    for e in pyfmmvec
]

exafmm_multipole_expansions = [
    np.array(tree.nodes[0].up_equiv) for tree in exafmmtreevec
]

In [17]:
# Create spheres centered on the root node with radii such that
# the root node's multipole expansion is valid
npoints = 1000
r0 = pyfmmvec[0].r0
x0 = pyfmmvec[0].x0

# Start of far-field
r = r0*3
c = x0
rvec = np.linspace(r, 10*r, 10)
spherevec = [sphere_data(npoints, r, c) for r in rvec]


# Store results for each experiment
pyfmm_results = [[] for i in range(len(pyfmmvec))]
exafmm_results = [[] for i in range(len(pyfmmvec))]
direct_results = [[] for i in range(len(pyfmmvec))]


# Compare expansion results with direct computation at spheres of increasing radius
# surrounding the upward equivalent surface of the root node.
for e_idx in range(len(pyfmmvec)):
    for i, sphere in enumerate(spherevec):

        pyfmm_results[e_idx].append(
            laplace_p2p_serial(
                sources=upward_equivalent_surfaces[e_idx],
                targets=sphere,
                source_densities=pyfmm_multipole_expansions[e_idx]
            )
        )

        exafmm_results[e_idx].append(
            laplace_p2p_serial(
                sources=upward_equivalent_surfaces[e_idx],
                targets=sphere,
                source_densities=exafmm_multipole_expansions[e_idx]
            )
        )

        direct_results[e_idx].append(
            laplace_p2p_serial(
                sources=pyfmmvec[e_idx].sources,
                targets=sphere,
                source_densities=pyfmmvec[e_idx].source_densities
            )
        )
        
pyfmm_results = np.array(pyfmm_results)
exafmm_results = np.array(exafmm_results)
direct_results = np.array(direct_results)

In [18]:
# Compute relative errors of multipole expansions of PyFMM and ExaFMM-T
pyfmm_relative_errors = [[] for i in range(len(pyfmmvec))]
exafmm_relative_errors = [[] for i in range(len(pyfmmvec))]

for e_idx in range(len(pyfmmvec)):
    for sph_idx in range(len(spherevec)):
        direct = direct_results[e_idx][sph_idx]
        pyfmm_estimate = pyfmm_results[e_idx][sph_idx]
        exafmm_estimate = exafmm_results[e_idx][sph_idx]
        
        pyfmm_err = abs(direct-pyfmm_estimate)/direct
        exafmm_err = abs(direct-exafmm_estimate)/direct

        pyfmm_relative_errors[e_idx].append(np.mean(pyfmm_err))
        exafmm_relative_errors[e_idx].append(np.mean(exafmm_err))
        
pyfmm_relative_errors = np.array(pyfmm_relative_errors)
exafmm_relative_errors = np.array(exafmm_relative_errors)

In [19]:
fig, ax = plt.subplots()
fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)

lines = ['solid', 'dashed', 'dashdot', 'dotted', 'solid', 'dashed']
markers = ['.', 'o', 'v', '+', 'D', 's']

for e_idx in range(len(pyfmmvec)):
    ax.semilogy(
        rvec, 
        pyfmm_relative_errors[e_idx], 
        marker=markers[e_idx], 
        linestyle=lines[e_idx],
        linewidth='0.4',
        markersize=2
    )

# Label with number of coefficients rather than expansion order
legend_labels = [6*(order-1)**2 + 2 for order in Evec]
plt.legend(legend_labels)
plt.xlabel('$R_s$')
plt.ylabel('Relative Error $\epsilon_{rel}$')
fig.set_size_inches(width, height)
fp = FIGURE_SAVEPATH  + '/pyexafmm_multipole_convergence.pdf'
plt.savefig(fp)

In [20]:
fig, ax = plt.subplots()
fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)

lines = ['solid', 'dashed', 'dashdot', 'dotted', 'solid', 'dashed']
markers = ['.', 'o', 'v', '+', 'D', 's']

for e_idx in range(len(pyfmmvec)):
    ax.semilogy(
        rvec, 
        exafmm_relative_errors[e_idx], 
        marker=markers[e_idx], 
        linestyle=lines[e_idx],
        linewidth='0.4',
        markersize=2
    )

# Label with number of coefficients
legend_labels = [ 6*(order-1)**2 + 2 for order in Evec]
plt.legend(legend_labels)
plt.xlabel('$R_s$')
plt.ylabel('Relative Error $\epsilon_{rel}$')
fig.set_size_inches(width, height)
fp = FIGURE_SAVEPATH  + '/exafmm_multipole_convergence.pdf'
plt.savefig(fp)
plt.show()

### Check convergence of local expansion of a given leaf node

In [21]:
# generate random charges for sources
e = Fmm('test')

# Displace points to +ve axis for simple conversion to Morton key representation in PyExaFMM
points = e.sources + 0.5
src_charges = e.source_densities

# create a list of source instances
sources = laplace.init_sources(points.astype(np.float32), src_charges)

# create a list of target instances
targets = laplace.init_targets(points)

fmm = laplace.LaplaceFmm(p=e.config['order_equivalent'], ncrit=150, filename="test_file.dat")
extree = laplace.setup(sources, targets, fmm)
depth = np.max([node.level for node in extree.nodes])

d = np.array([extree.nodes[0].x[0], extree.nodes[0].x[1], extree.nodes[0].x[2]]) + extree.nodes[0].r

exanchors = [
    np.round(np.array([(n.x[0]-n.r)/(1/2**n.level), (n.x[1]-n.r)/(1/2**n.level), (n.x[2]-n.r)/(1/2**n.level), n.level])).astype(np.int64)
    for n in extree.leafs
] 

exleaves = np.array([morton.encode_anchor(a) for a in exanchors])

# exanchors = [
#     np.round(np.array([(n.x[0]-n.r)/(1/2**n.level), (n.x[1]-n.r)/(1/2**n.level), (n.x[2]-n.r)/(1/2**n.level), n.level])).astype(np.int64)
#     for n in extree.nodes
# ]

# exnodes =  np.array([morton.encode_anchor(a) for a in exanchors])

# exnodes.sort()
# exleaves.sort()

In [22]:
def find_vertices(anchor, r0, x0):
    
    level = anchor[-1]
    scale = 1./(1 << level)
    r = r0*scale
    
    vertices = np.array([
        [0, 0, 0],
        [1, 0, 0],
        [1, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [1, 0, 1],
        [1, 1, 1],
        [0, 1, 1]
    ]) + anchor[:3]
    
    vertices = vertices*scale + (x0-r0)

    return vertices, ([
        [vertices[0], vertices[1], vertices[2], vertices[3]],
        [vertices[4], vertices[5], vertices[6], vertices[7]], 
        [vertices[0], vertices[1], vertices[5], vertices[4]], 
        [vertices[2], vertices[3], vertices[7], vertices[6]], 
        [vertices[1], vertices[2], vertices[6], vertices[5]],
        [vertices[4], vertices[7], vertices[3], vertices[0]]
    ])


def plot_node(ax, key, r0, x0, c='gray'):
    anchor = morton.decode_key(key)
    vertices, edges = find_vertices(anchor, r0, x0)
    zline = vertices[:, 2]
    xline = vertices[:, 0]
    yline = vertices[:, 1]
    ax.scatter3D(xline, yline, zline, c='gray')
    ax.add_collection3d(Poly3DCollection(edges, facecolor=c, edgecolors='black', alpha=0.1))

In [23]:
leaf_idx = 34
node = extree.leafs[leaf_idx]

trg_values = laplace.evaluate(extree, fmm)

In [24]:
# e = Fmm('test')
e.run()

In [59]:
idx = np.where(e.complete == exleaves[leaf_idx])[0][0]
local_expansion = e.local_expansions[
    e.key_to_index[exleaves[leaf_idx]]*e.nequivalent_points:
    e.key_to_index[exleaves[leaf_idx]]*e.nequivalent_points+e.nequivalent_points
]

targets = e.targets[
    e.target_index_pointer[
        e.key_to_leaf_index[exleaves[leaf_idx]]:e.key_to_leaf_index[exleaves[leaf_idx+1]]
    ]
]

In [67]:
# Start of local-field
c = morton.find_physical_center_from_key(exleaves[leaf_idx], x0, r0)
r = r0 / (1 << morton.find_level(exleaves[leaf_idx]))

downward_equivalent_surface = scale_surface(
    e.equivalent_surface, 
    e.r0, 
    morton.find_level(exleaves[leaf_idx]),
    c, 
    e.alpha_outer
)

rvec = np.linspace(0.1*r, 0.01*r, 3)
spherevec = [sphere_data(npoints, r, c) for r in rvec]


# Compare expansion results with direct computation at spheres of decreasing radius
# inside the downward equivalent surface of a given leaf node.
pyfmm_results = []
exafmm_results = []
direct_results = []

for i, sphere in enumerate(spherevec):

    pyfmm_results.append(
        laplace_p2p_serial(
            sources=downward_equivalent_surface,
            targets=targets,
            source_densities=local_expansion
        )
    )
    
    exafmm_results.append(
        laplace_p2p_serial(
            sources=downward_equivalent_surface,
            targets=targets,
            source_densities=np.array(extree.leafs[leaf_idx].dn_equiv, np.float32)
        )
    )
    direct_results.append()

pyfmm_results = np.array(pyfmm_results)
exafmm_results = np.array(exafmm_results)

In [69]:
exafmm_results

array([[183.68677],
       [183.68677],
       [183.68677]], dtype=float32)

In [68]:
pyfmm_results

array([[162.1432],
       [162.1432],
       [162.1432]], dtype=float32)

### Convergence of Potentials

In [70]:
# Calculate potential directly
direct_potential = laplace_p2p_serial(
    sources=pyfmmvec[0].sources,
    targets=pyfmmvec[0].targets, 
    source_densities=pyfmmvec[0].source_densities
)

In [75]:
# Store results for each experiment
pyfmm_potential_errors = []
exafmm_potential_errors = []


# Compare expansion results with direct computation at spheres of increasing radius
# surrounding the upward equivalent surface of the root node.
for e_idx in range(len(pyfmmvec)):
    pyfmm_estimate = pyfmmvec[e_idx].target_potentials
    pyfmm_error = np.mean(abs(direct_potential-pyfmm_estimate[:,0])/direct_potential)
    pyfmm_potential_errors.append(pyfmm_error)
    
    exafmm_estimate = ex_target_potentials[e_idx]
    exafmm_error = np.mean(abs(direct_potential-exafmm_estimate[:,0])/direct_potential)
    exafmm_potential_errors.append(exafmm_error)

In [78]:
fig, ax = plt.subplots()
fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)

ax.semilogy(
    Evec, 
    exafmm_potential_errors, 
    marker='o', 
    linestyle='--',
    linewidth='0.4',
    markersize=2
)


ax.semilogy(
    Evec, 
    pyfmm_potential_errors, 
    marker='x', 
    linestyle=':',
    linewidth='0.4',
    markersize=2
)


# Label with number of coefficients
legend_labels = ['ExaFMM', 'PyExaFMM']

plt.legend(legend_labels)
plt.xlabel('Order of Expansions')
plt.ylabel('Relative Error $\epsilon_{rel}$')
fig.set_size_inches(width, height)
fp = FIGURE_SAVEPATH  + '/potential_convergence.pdf'
plt.savefig(fp)
plt.show()